In [ ]:
import torch
import torch.nn as nn
class HybridAutoencoder(nn.Module):
    def __init__(self):
        super(HybridAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(8, 10),
            nn.LeakyReLU(negative_slope=0.5),
            nn.Linear(10, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 10),
            nn.LeakyReLU(negative_slope=0.5),
            nn.Linear(10, 8),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
model = HybridAutoencoder()

model_path = '/content/drive/MyDrive/SecurityApplicationProject/hybrid_autoencoder_parameters.pth'
model.load_state_dict(torch.load(model_path))

model.eval()


HybridAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=8, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.5)
    (2): Linear(in_features=10, out_features=8, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.5)
    (2): Linear(in_features=10, out_features=8, bias=True)
    (3): Sigmoid()
  )
)

In [ ]:
test_input = torch.tensor([[0, 1, 0, 1, 1, 0, 1, 0]]).float()
with torch.no_grad():
    encrypted_vector = model.encoder(test_input)
    print('Encrypted vector:', encrypted_vector.numpy())


decrypted_vector = model.decoder(encrypted_vector)
binarized_output = torch.where(decrypted_vector > 0.5, torch.tensor(1.0), torch.tensor(0.0))
print('Binarized output:', binarized_output)

correct_predictions = (binarized_output == test_input).float()
accuracy = correct_predictions.mean()
print(f'Accuracy: {accuracy.item() * 100}%')
print(f'Correct predictions: {correct_predictions.sum().item()} out of {test_input.nelement()}')
comparison = torch.cat((test_input, binarized_output, correct_predictions), 0)
print('Row 1: Original Input\nRow 2: Binarized Output\nRow 3: Correct Predictions (1: Correct, 0: Incorrect)\n', comparison)



Encrypted vector: [[ 0.02202405  1.4976857  -1.5635588  -2.3500974  -1.4465282   9.93152
  -3.9664261  -3.7735186 ]]
Binarized output: tensor([[0., 1., 0., 1., 1., 0., 1., 0.]])
Accuracy: 100.0%
Correct predictions: 8.0 out of 8
Row 1: Original Input
Row 2: Binarized Output
Row 3: Correct Predictions (1: Correct, 0: Incorrect)
 tensor([[0., 1., 0., 1., 1., 0., 1., 0.],
        [0., 1., 0., 1., 1., 0., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


In [ ]:
def load_model(model_path):
    model = HybridAutoencoder()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def interactive_test(model):
    try:
        user_input = input("Enter an 8-bit binary sequence (e.g., 01010101): ")
        test_input = torch.tensor([[int(bit) for bit in user_input]]).float()
        with torch.no_grad():
            encrypted_vector = model.encoder(test_input)
            print('Encrypted vector:', encrypted_vector.numpy())

            decrypted_vector = model.decoder(encrypted_vector)
            binarized_output = torch.where(decrypted_vector > 0.5, torch.tensor(1.0), torch.tensor(0.0))
            print('Binarized output:', binarized_output.numpy())

    except ValueError:
        print("Invalid input. Please ensure your input is an 8-bit binary sequence.")

def string_to_ascii_binary(input_string):
    """Convert a string to a list of 8-bit binary representations of its ASCII characters."""
    return [format(ord(c), '08b') for c in input_string]

def binary_to_tensor(binary_list):
    """Convert a list of 8-bit binary strings to a tensor."""
    return torch.tensor([[int(bit) for bit in binary] for binary in binary_list], dtype=torch.float)

def tensor_to_binary(tensor):
    """Convert a tensor to a list of binary strings."""
    return [''.join(str(int(bit)) for bit in sequence) for sequence in tensor]

def binary_to_ascii(binary_list):
    """Convert a list of binary strings to their ASCII character equivalents."""
    return ''.join([chr(int(binary, 2)) for binary in binary_list])

if __name__ == "__main__":
    model_path = '/content/drive/MyDrive/SecurityApplicationProject/hybrid_autoencoder_parameters.pth'
    model = load_model(model_path)

    user_input = input("Enter a passage in English: ")
    ascii_binaries = string_to_ascii_binary(user_input)
    input_tensor = binary_to_tensor(ascii_binaries)

    encrypted_vectors = []
    with torch.no_grad():
        for sequence in input_tensor:
            encrypted_vector = model.encoder(sequence.unsqueeze(0))
            encrypted_vectors.append(encrypted_vector)

    decrypted_vectors = []
    for vector in encrypted_vectors:
        decrypted_vector = model.decoder(vector)
        binarized_output = torch.where(decrypted_vector > 0.5, torch.tensor(1.0), torch.tensor(0.0))
        decrypted_vectors.append(binarized_output)

    decrypted_binaries = tensor_to_binary(torch.cat(decrypted_vectors))
    decrypted_text = binary_to_ascii(decrypted_binaries)

    print(f"Original Text: {user_input}")
    print(f"Decrypted Text: {decrypted_text}")


Enter a passage in English: IITJ
Original Text: IITJ
Decrypted Text: IITJ


In [ ]:
def encrypted_vectors_to_string(encrypted_vectors):
    """Convert encrypted vectors to a string representation."""
    # Convert each tensor in encrypted_vectors to a string of numbers separated by commas
    # and join different vectors with a semicolon for readability
    return '; '.join([', '.join([f'{element:.4f}' for element in vector.squeeze().tolist()]) for vector in encrypted_vectors])

if __name__ == "__main__":
    model_path = '/content/drive/MyDrive/SecurityApplicationProject/hybrid_autoencoder_parameters.pth'
    model = load_model(model_path)

    user_input = input("Enter a passage in English: ")
    ascii_binaries = string_to_ascii_binary(user_input)
    input_tensor = binary_to_tensor(ascii_binaries)

    encrypted_vectors = []
    with torch.no_grad():
        for sequence in input_tensor:
            encrypted_vector = model.encoder(sequence.unsqueeze(0))
            encrypted_vectors.append(encrypted_vector)

    # Convert encrypted vectors to a string for display
    encrypted_string = encrypted_vectors_to_string(encrypted_vectors)

    decrypted_vectors = []
    for vector in encrypted_vectors:
        decrypted_vector = model.decoder(vector)
        binarized_output = torch.where(decrypted_vector > 0.5, torch.tensor(1.0), torch.tensor(0.0))
        decrypted_vectors.append(binarized_output)

    decrypted_binaries = tensor_to_binary(torch.cat(decrypted_vectors))
    decrypted_text = binary_to_ascii(decrypted_binaries)

    print(f"Original Text: {user_input}")
    print(f"Encrypted (Vector Format): {encrypted_string}")
    print(f"Decrypted Text: {decrypted_text}")


Enter a passage in English: IITJ
Original Text: IITJ
Encrypted (Vector Format): -2.0258, 2.2443, -1.5781, 1.3165, 2.6042, 4.7823, -1.7187, -1.1811; -2.0258, 2.2443, -1.5781, 1.3165, 2.6042, 4.7823, -1.7187, -1.1811; 5.0964, 0.2891, -5.8141, -3.8853, 2.7905, 2.7452, 2.6121, -1.4434; -3.6573, 3.2207, -2.3393, 0.0176, -2.4937, 5.7965, -6.2938, -2.9294
Decrypted Text: IITJ


In [ ]:
def string_to_encrypted_vectors(encrypted_string):
    """Parse the encrypted string back into a list of tensors."""
    vector_strings = encrypted_string.split('; ')
    vectors = [torch.tensor([float(num) for num in vector.split(', ')], dtype=torch.float).unsqueeze(0) for vector in vector_strings]
    return vectors

def decrypt_encrypted_string(model, encrypted_string):
    """Decrypt an encrypted string back to text."""
    encrypted_vectors = string_to_encrypted_vectors(encrypted_string)
    decrypted_vectors = []
    with torch.no_grad():
        for vector in encrypted_vectors:
            decrypted_vector = model.decoder(vector)
            binarized_output = torch.where(decrypted_vector > 0.5, torch.tensor(1.0), torch.tensor(0.0))
            decrypted_vectors.append(binarized_output)
    decrypted_binaries = tensor_to_binary(torch.cat(decrypted_vectors))
    decrypted_text = binary_to_ascii(decrypted_binaries)
    return decrypted_text

if __name__ == "__main__":
    model_path = '/content/drive/MyDrive/SecurityApplicationProject/hybrid_autoencoder_parameters.pth'
    model = load_model(model_path)

    # Assuming you have an encrypted string (from previous steps or stored separately)
    encrypted_string = input("Encrypted string: ") # Replace with your actual encrypted string
    decrypted_text = decrypt_encrypted_string(model, encrypted_string)

    print(f"Decrypted Text from Encrypted String: {decrypted_text}")


Encrypted string: -2.0258, 2.2443, -1.5781, 1.3165, 2.6042, 4.7823, -1.7187, -1.1811; -2.0258, 2.2443, -1.5781, 1.3165, 2.6042, 4.7823, -1.7187, -1.1811; 5.0964, 0.2891, -5.8141, -3.8853, 2.7905, 2.7452, 2.6121, -1.4434; -3.6573, 3.2207, -2.3393, 0.0176, -2.4937, 5.7965, -6.2938, -2.9294
Decrypted Text from Encrypted String: IITJ
